In [5]:
# Smile Detection - Multi-Camera, Movable Window Version

import cv2
import numpy as np
import mediapipe as mp
import threading
import time
import ctypes
from tensorflow.keras.models import load_model

# Load model and initialize MediaPipe3331`11
model = load_model(r'C:\Users\Home\Downloads\BestModelsElimination\BestModelsElimination\final_smile_detection_model-v2_Trainedw_DiffNN.keras')
mp_face_detection = mp.solutions.face_detection
face_detection = mp_face_detection.FaceDetection(min_detection_confidence=0.5)

# Get screen resolution
user32 = ctypes.windll.user32
screen_width, screen_height = user32.GetSystemMetrics(0), user32.GetSystemMetrics(1)

def adjust_brightness(frame, alpha=1.2, beta=30):
    return cv2.convertScaleAbs(frame, alpha=alpha, beta=beta)

class VideoStream:
    def __init__(self, src=0):
        self.src = src
        self.capture = cv2.VideoCapture(src, cv2.CAP_DSHOW)
        self.capture.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
        self.capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
        self.status = False
        self.frame = None
        self.thread = threading.Thread(target=self.update, daemon=True)
        self.thread.start()

    def update(self):
        while True:
            if self.capture.isOpened():
                self.status, self.frame = self.capture.read()

    def get_frame(self):
        return self.frame if self.status else None

    def stop(self):
        self.capture.release()

    def switch_camera(self, new_src):
        self.stop()
        self.__init__(new_src)

# Start with second webcam (index 1, assuming built-in cam is 0)
camera_index = 1
stream = VideoStream(camera_index)
prev_time = time.time()

# Create movable resizable window
cv2.namedWindow("Smile Detection", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Smile Detection", screen_width, screen_height)
cv2.moveWindow("Smile Detection", 100, 100)  # Start window not maximized or locked

def process_frame(frame):
    frame = adjust_brightness(frame)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_detection.process(rgb_frame)

    if results.detections:
        for detection in results.detections:
            bbox = detection.location_data.relative_bounding_box
            ih, iw, _ = frame.shape
            x, y, w, h = (
                int(bbox.xmin * iw),
                int(bbox.ymin * ih),
                int(bbox.width * iw),
                int(bbox.height * ih)
            )
            x, y, w, h = max(0, x), max(0, y), min(iw - x, w), min(ih - y, h)
            face_crop = frame[y:y+h, x:x+w]
            if face_crop.size == 0:
                continue

            face_resized = cv2.resize(face_crop, (150, 150))
            face_array = face_resized.astype("float32") / 255.0
            face_array = np.expand_dims(face_array, axis=0)

            prediction = model.predict(face_array, verbose=0)
            confidence = prediction[0][0]

            # Improved margin logic
            if confidence > 0.55:
                label = "Smile"
            elif confidence < 0.45:
                label = "Not Smile"
            elif 0.52 < confidence <= 0.55:
                label = "Possible Smile"
            elif 0.45 <= confidence < 0.48:
                label = "Possible Not Smile"
            else:
                label = "Undecided"

            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, f"{label} ({confidence:.2f})", (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)
    return frame

try:
    while True:
        frame = stream.get_frame()
        if frame is None:
            continue

        processed_frame = process_frame(frame)

        curr_time = time.time()
        fps = 1 / (curr_time - prev_time)
        prev_time = curr_time
        cv2.putText(processed_frame, f"FPS: {int(fps)}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

        display_frame = cv2.resize(processed_frame, (screen_width, screen_height))
        cv2.imshow("Smile Detection", display_frame)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break
        elif key == ord('1'):
            camera_index = 0
            print("Switching to Camera 0 (likely laptop webcam)")
            stream.switch_camera(camera_index)
        elif key == ord('2'):
            camera_index = 1
            print("Switching to Camera 1 (first USB webcam)")
            stream.switch_camera(camera_index)
        elif key == ord('3'):
            camera_index = 2
            print("Switching to Camera 2 (second USB webcam)")
            stream.switch_camera(camera_index)

except KeyboardInterrupt:
    print("Stream interrupted by user.")

finally:
    stream.stop()
    cv2.destroyAllWindows()
    print("Stream stopped.")


ModuleNotFoundError: No module named 'cv2'